In [107]:
import numpy as np

In [108]:
size = 10
deck = np.arange(size).reshape(size)
print(deck)

[0 1 2 3 4 5 6 7 8 9]


In [109]:
# 1. deal into new stack
def shuffle1(deck):
    #newdeck = np.copy(deck)
    newdeck = deck[::-1] 
    return newdeck

print(shuffle1(deck))

[9 8 7 6 5 4 3 2 1 0]


In [110]:
# 2. cut N cards (N can be negative)
def shuffle2(deck,N):
    if N>0:
        cut = deck[0:N]
        left = deck[N:]
        newdeck = np.append(left,cut)
    elif N<0:
        cut = deck[len(deck)+N:]
        left = deck[:N]
        newdeck = np.append(cut,left)
    return newdeck
    
print(shuffle2(deck,-4))

[6 7 8 9 0 1 2 3 4 5]


In [166]:
# 3. deal with increment N
def shuffle3(deck,N):
    newdeck = np.zeros(len(deck))
    j = 0
    for i in range(len(deck)):
        #print(j)
        newdeck[j] = deck[i]
        j = (j+N)%len(deck)
    return newdeck
 
deck = np.arange(size).reshape(size)
print(shuffle3(deck,3))

[    0.  3336.  6672. ... 10006.  3335.  6671.]


In [167]:
lines = [
'deal with increment 7',
'deal into new stack',
'deal into new stack']
# Result: 0 3 6 9 2 5 8 1 4 7 --> OK

lines = [
'cut 6',
'deal with increment 7',
'deal into new stack' ] 
#Result: 3 0 7 4 1 8 5 2 9 6 --> OK

lines = [
'deal with increment 7',
'deal with increment 9',
'cut -2' ]
#Result: 6 3 0 7 4 1 8 5 2 9 --> OK

lines = [
'deal into new stack',
'cut -2',
'deal with increment 7',
'cut 8',
'cut -4',
'deal with increment 7',
'cut 3',
'deal with increment 9',
'deal with increment 3',
'cut -1' ]
# Result: 9 2 5 8 1 4 7 0 3 6 --> OK

size = 10

In [184]:
with open("input22.txt") as f:
    lines = [l.rstrip('\n') for l in f]
#lines

size = 10007

In [185]:
s1 = 'deal into new stack'
s2 = 'cut '
s3 = 'deal with increment '

deck = np.arange(size).reshape(size)

for l in lines:
    if l == s1:
        deck = shuffle1(deck)
    elif l[0:4]== s2:
        n = l.split(' ')
        N = int(n[1])
        deck = shuffle2(deck,N)
    else:
        n = l.split(' ')
        N = int(n[3])
        deck = shuffle3(deck,N)

In [186]:
card = 2019
pos = np.where(deck==card)[0][0]
print("Card",card,"is found at position",pos)

Card 2019 is found at position 7171
